In [1]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline


if torch.cuda.is_available():
    device = torch.device("cuda") 
    !nvidia-smi
else:
    device = torch.device("cpu")

print(f'Using {device}')

Wed Jan  3 01:20:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050      WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P0              N/A / ERR! |      0MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
! pip install timm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from torchvision.models.feature_extraction import create_feature_extractor

model_type = "GPUNet-0" # select one from above
precision = "fp32"
gpunet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_gpunet', pretrained=True, model_type=model_type, model_math=precision)
# gpunet = 


Using cache found in C:\Users\Maryk/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


model_type= GPUNet-0
mapped model latency= 0.65ms
testing:  batch= 1  latency= 0.65ms  gpu= GV100


175259648B [25:23, 115015.24B/s]                                 


In [11]:
PATH = '../Models/gpunet.pth'

torch.save(gpunet,PATH)

In [18]:
from torch import load
gg = load('D:/Project/YOLO v6/YOLOv6/Models/gpunet.pth')

In [20]:
gg

GPUNet(
  (network): Sequential(
    (head: 2): prologue_i3_o32_s2_swish
    (stage: 1 layer3): conv_k3_i32_o32_s1_relu
    (stage: 1 layer4): conv_k3_i32_o32_s1_relu
    (stage: 2 layer5): er_k3_e5_i32_o32_s2_relu_se_False
    (stage: 2 layer6): er_k3_e5_i32_o32_s1_relu_se_False
    (stage: 3 layer7): er_k3_e5_i32_o64_s2_relu_se_False
    (stage: 3 layer8): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 3 layer9): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 4 layer10): irb_k3_e5_i64_o256_s2_swish_se_False
    (stage: 4 layer11): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 4 layer12): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 6 layer13): irb_k3_e5_i256_o704_s2_relu_se_True
    (stage: 6 layer14): irb_k3_e5_i704_o704_s1_relu_se_True
    ( layer15): epilogue_i704_o1280_s1_relu
  )
)

In [26]:
# for name, layer in gpunet.named_modules():
#     print(name)

import torch.nn as nn

gpunet.to(device)
gpunet.eval()

for name, layer in gpunet.named_modules():
    if isinstance(layer, nn.ReLU):
        print(name, layer)

network.stage: 1 layer3.act1 ReLU(inplace=True)
network.stage: 1 layer4.act1 ReLU(inplace=True)
network.stage: 2 layer5.act1 ReLU(inplace=True)
network.stage: 2 layer6.act1 ReLU(inplace=True)
network.stage: 3 layer7.act1 ReLU(inplace=True)
network.stage: 3 layer8.act1 ReLU(inplace=True)
network.stage: 3 layer9.act1 ReLU(inplace=True)
network.stage: 6 layer13.act1 ReLU(inplace=True)
network.stage: 6 layer13.act2 ReLU(inplace=True)
network.stage: 6 layer13.se.act1 ReLU(inplace=True)
network.stage: 6 layer14.act1 ReLU(inplace=True)
network.stage: 6 layer14.act2 ReLU(inplace=True)
network.stage: 6 layer14.se.act1 ReLU(inplace=True)
network. layer15.net.2 ReLU(inplace=True)


In [67]:
return_nodes = {
    "layer15.net.2": "layer14"
}
model2 = create_feature_extractor(gpunet, eval_return_nodes={"layer4":"0"} , train_return_nodes={})
# intermediate_outputs = model2(x)
# gpunet.modules

TypeError: rand() received an invalid combination of arguments - got (Proxy, device=Attribute, dtype=Attribute), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [74]:
from torchvision.models._utils import IntermediateLayerGetter
model2 = create_feature_extractor(gpunet, return_nodes={"layer4":"0"})

AttributeError: 'generator' object has no attribute 'training'

In [27]:
import torchvision
model = torchvision.models.resnet50(pretrained=True)


In [33]:
model.layer4

AttributeError: 'Sequential' object has no attribute 'conv2'

In [122]:
# mds = list(gpunet.modules())
mdl = mds[2]
print(mdl)
create_feature_extractor((mdl), return_nodes=[]) 

prologue_i3_o32_s2_swish


Prologue()

In [108]:
c = 0
for i in mds:
    print(c)
    c+= 1
    print(i)

0
GPUNet(
  (network): Sequential(
    (head: 2): prologue_i3_o32_s2_swish
    (stage: 1 layer3): conv_k3_i32_o32_s1_relu
    (stage: 1 layer4): conv_k3_i32_o32_s1_relu
    (stage: 2 layer5): er_k3_e5_i32_o32_s2_relu_se_False
    (stage: 2 layer6): er_k3_e5_i32_o32_s1_relu_se_False
    (stage: 3 layer7): er_k3_e5_i32_o64_s2_relu_se_False
    (stage: 3 layer8): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 3 layer9): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 4 layer10): irb_k3_e5_i64_o256_s2_swish_se_False
    (stage: 4 layer11): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 4 layer12): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 6 layer13): irb_k3_e5_i256_o704_s2_relu_se_True
    (stage: 6 layer14): irb_k3_e5_i704_o704_s1_relu_se_True
    ( layer15): epilogue_i704_o1280_s1_relu
  )
)
1
Sequential(
  (head: 2): prologue_i3_o32_s2_swish
  (stage: 1 layer3): conv_k3_i32_o32_s1_relu
  (stage: 1 layer4): conv_k3_i32_o32_s1_relu
  (stage: 2 layer5): er_k3_e5_i32_o32_s2_relu_se_

In [116]:
c = 0
for i in model.modules():
    print(c)
    c+= 1
    print(i)

0
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1

use hook forward

In [123]:
def get_features(features ,name):
    def hook(model, input, output):
        res = output.detach()
        if name == "c3":
            target = torch.zeros(res.shape[0] , res.shape[1] , 80 , 80)
            target[:, :, :79,:79] = res
            features[name] = target
        
        else : features[name] = res
    return hook

In [35]:
features = {}
mds = list(gpunet.modules())
c3 = mds[79]
c4 = mds[93]
c5 = mds[113]

c3.register_forward_hook(get_features(features,"c3"))
c4.register_forward_hook(get_features(features,"c4"))
c5.register_forward_hook(get_features(features,"c5"))

In [45]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
uris = [
    'http://images.cocodataset.org/test-stuff2017/000000024309.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000028117.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000006149.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000004954.jpg',
]

batch = torch.cat(
    [utils.prepare_input_from_uri(uri) for uri in uris]
)

if precision == "fp16":
    batch = batch.half()
    
print("Ready to run inference...")


Using cache found in C:\Users\Maryk/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


Ready to run inference...


In [44]:
features

{}

In [43]:


gpunet(batch)


tensor([[-0.1414,  0.2187, -0.1323,  ..., -1.3768, -0.6344, -1.4868],
        [-0.9334, -0.7336,  0.7334,  ..., -1.6523,  0.4345,  0.5191],
        [ 0.1460, -0.6631,  1.3636,  ...,  0.0372, -0.3004, -0.5929],
        [-2.2497, -1.1935, -0.3671,  ..., -0.9032, -0.9653, -0.9141]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [145]:
FEATS = np.concatenate(FEATS)
print(FEATS.shape)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 4) + inhomogeneous part.

In [146]:
FEATS

[[array([[ 0.2259801 , -0.15294072, -1.512263  , ..., -1.2224617 ,
          -0.90127724,  0.02961297],
         [ 0.09717762,  0.67677516,  0.5848702 , ...,  0.15049517,
           1.4610047 , -0.17677425],
         [ 0.6227139 ,  0.1135464 ,  0.521129  , ..., -1.1161091 ,
          -0.3224512 , -1.0071607 ],
         [ 0.07814017,  0.33386844,  1.6657013 , ...,  0.2418642 ,
           1.3473537 ,  1.6984595 ]], dtype=float32),
  array([[[[-2.16793942e+00, -1.92733169e+00,  6.87612653e-01, ...,
             1.31629989e-01, -1.93996787e+00, -1.28968441e+00],
           [-7.63811231e-01, -1.37549326e-01, -8.67556036e-01, ...,
            -1.15073383e+00, -9.61326540e-01, -2.33602393e-02],
           [ 7.05757663e-02, -5.99115491e-01, -1.92986655e+00, ...,
            -1.91740251e+00, -1.47812116e+00, -1.20598562e-01],
           ...,
           [ 9.21590030e-01,  2.42263332e-01, -5.06356478e-01, ...,
             4.87339687e+00,  7.50002205e-01, -1.07349372e-02],
           [ 2.16605115

In [25]:


import torchvision

gpunet = torch.load('../Models/gpunet.pth')
# gpunet = torch.load('../Models/gpunet.pth')

ModuleNotFoundError: No module named 'PyTorch'

In [76]:
gpunet

GPUNet(
  (network): Sequential(
    (head: 2): prologue_i3_o32_s2_swish
    (stage: 1 layer3): conv_k3_i32_o32_s1_relu
    (stage: 1 layer4): conv_k3_i32_o32_s1_relu
    (stage: 2 layer5): er_k3_e5_i32_o32_s2_relu_se_False
    (stage: 2 layer6): er_k3_e5_i32_o32_s1_relu_se_False
    (stage: 3 layer7): er_k3_e5_i32_o64_s2_relu_se_False
    (stage: 3 layer8): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 3 layer9): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 4 layer10): irb_k3_e5_i64_o256_s2_swish_se_False
    (stage: 4 layer11): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 4 layer12): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 6 layer13): irb_k3_e5_i256_o704_s2_relu_se_True
    (stage: 6 layer14): irb_k3_e5_i704_o704_s1_relu_se_True
    ( layer15): epilogue_i704_o1280_s1_relu
  )
)

In [107]:
import pickle
import torchvision
# model = torchvision.models.resnet50()
pickle.dump(features, open('features.pkl', 'wb'))

In [108]:
f = pickle.load(open('features.pkl', 'rb'))

In [81]:
f

{'c3': tensor([[[[-1.4475e+00, -6.0274e-01, -1.1192e+00,  ..., -1.1772e+00,
             8.5668e-01, -4.0054e+00],
           [ 5.5427e+00,  7.4018e+00,  6.3426e+00,  ...,  1.1055e+00,
             2.7249e+00,  7.9513e-01],
           [-2.0911e+00,  2.0776e+00,  3.5489e+00,  ...,  5.4108e-01,
            -6.1577e-02, -5.0017e+00],
           ...,
           [-3.2619e+00,  2.2600e+00,  4.9699e+00,  ..., -5.3592e-01,
             1.5133e+00, -2.4226e+00],
           [-2.8329e+00,  2.4312e+00,  4.3236e+00,  ...,  8.2488e-01,
             3.6726e+00,  1.0772e+00],
           [-5.5783e+00, -4.7149e-01, -1.1500e+00,  ..., -9.9162e-01,
             5.5484e-03, -4.5060e+00]],
 
          [[ 3.2328e+00,  1.0024e+00,  4.9144e+00,  ...,  2.4363e+00,
            -9.1852e-01, -1.8235e+00],
           [ 6.3109e+00,  5.3709e+00,  7.7181e+00,  ..., -1.4394e+00,
            -2.0081e+00, -4.2650e+00],
           [ 3.3407e+00,  3.1075e+00,  6.0657e-01,  ...,  1.4874e+00,
            -3.6912e+00, -1.2811e

In [8]:
def get_features(features,name):
    def hook(model, input, output):
        features[name] = output.detach()

    return hook

In [99]:
from collections import OrderedDict
from typing import Dict, Callable
import torch

def remove_all_forward_hooks(model: torch.nn.Module) -> None:
    for name, child in model._modules.items():
        if child is not None:
            if hasattr(child, "_forward_hooks"):
                child._forward_hooks: Dict[int, Callable] = OrderedDict()
            remove_all_forward_hooks(child)
remove_all_forward_hooks(gpunet)

In [59]:
type(batch)

torch.Tensor

In [91]:
import torchvision.transforms as T


batch = torch.cat(
    [utils.prepare_input_from_uri(uri) for uri in uris]
).to(device)
prepbatch = T.Resize(640)(batch)

In [100]:
        mds = list(gpunet.modules())
        c3 = mds[34]
        c4 = mds[68]
        c5 = mds[107]
        # c3 = self.model.layer2
        # c4 = self.model.layer3
        # c5 = self.model.layer4
        features = {}

        hc3 = c3.register_forward_hook(get_features(features ,"c3"))
        hc4 = c4.register_forward_hook(get_features(features ,"c4"))
        hc5 = c5.register_forward_hook(get_features(features ,"c5"))
        

In [101]:
# x = np.zeros((32,3 , 3 ,3))
with torch.no_grad():
    gpunet(prepbatch)

In [106]:
print(features['c3'].shape)
print(features['c4'].shape)
print(features['c5'].shape)

torch.Size([4, 64, 80, 80])
torch.Size([4, 256, 40, 40])
torch.Size([4, 704, 20, 20])


In [103]:
## gpunet  (224)
## c3  256, 40, 40
## c4 704 20 20
## c5 1280 1 1
## out 1 1000

features['c3'].reshape(4 , 64 , 80 , 80)

RuntimeError: shape '[4, 64, 80, 80]' is invalid for input of size 1597696

In [105]:
target = torch.zeros(4 , 64 , 80 , 80)
target[:, :, :79,:79] = features['c3']
features['c3'] = target
